In [62]:
import gym
import matplotlib.pyplot as plt
import time
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [63]:
env = gym.make("CartPole-v1", render_mode='human') # render_mode='rgb_array'

In [64]:
## Observation and action space 
#obs_space = env.observation_space
#action_space = env.action_space
#print("The observation space: {}".format(obs_space))
#print("The action space: {}".format(action_space))

In [65]:
def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

In [5]:
num_steps = 1000
num_episodes = 10
totals = []
for episode in range(num_episodes):
    episode_reward = 0
    obs = env.reset()[0]
    for step in range(num_steps):
        action = basic_policy(obs)#env.action_space.sample()

        # app={ly the action
        obs, reward, terminated, truncated, info  = env.step(action)
        episode_reward += reward
        ## Render the env
        #env.render()
        
        # Wait a bit before the next frame unless you want to see a crazy fast video
        time.sleep(0.001)

        # If the epsiode is up, then start another one
        if terminated or truncated:
            break
    totals.append(episode_reward)

# Neural Network Policies

To choose an action, the network will first estimate a probability for each action, then select an action randomly according to the estimated probabilities.

In [7]:
# 1. Specify the network architecture
n_inputs = 4
n_hidden = 4
n_outputs = 1
initializer = tf.variance_scaling_initializer()

# 2. Build the neural network
X = tf.placeholder(tf.float32, shape = [None, n_inputs])
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
outputs = tf.layers.dense(hidden, n_outputs, activation=tf.nn.sigmoid, kernel_initializer=initializer)

# 3. Select a random action based on the estimated probabilities
p_left_and_right = tf.concat(axis = 1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples = 1)

init = tf.global_variables_initializer()

Randomly initialize policy neural network and use it to play few games

In [11]:
n_max_steps = 1000
n_games = 10

with tf.Session() as sess:
    init.run()
    for game in range(n_games):
        obs = env.reset(seed = 42)[0]
        for step in range(n_max_steps):
            #env.render()
            action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})

            obs, reward, terminated, truncated, info  = env.step(action_val[0][0])
            if terminated or truncated:
                break

2023-05-04 14:19:12.132093: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-04 14:19:12.132294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 14:19:12.132583: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 920MX computeCapability: 5.0
coreClock: 0.993GHz coreCount: 2 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 14.92GiB/s
2023-05-04 14:19:12.132620: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-05-04 14:19:12.132665: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-05-04 14:19:12.132687: I tensorflow/stream_executor/platform/defa

Learning the basic policy (go left if the pole is tilting left, and go right if it is tilting right)

In [13]:
reset_graph()

n_inputs = 4
n_hidden = 4
n_outputs = 1

learning_rate = 0.01

initializer = tf.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape = [None, n_inputs])
y = tf.placeholder(tf.float32, shape = [None, n_outputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs = tf.nn.sigmoid(logits) # probability of action 0 (left)

p_left_and_right = tf.concat(axis = 1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples = 1)

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(cross_entropy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [31]:
n_environments = 10
n_iterations = 1000
obs = env.reset(seed = 42)[0]

with tf.Session() as sess:
    init.run()

    for iteration in range(n_iterations):
        target_proba = [1.0] if obs[2] < 0 else [0.] # if angle<0 we want proba(left)=1., or else proba(left)=0.
    
        action_val, _ = sess.run([action, training_op], feed_dict={X: np.array([obs]), y: np.array([target_proba])})

        obs, reward, terminated, truncated, info  = env.step(action_val[0][0])
        if terminated or truncated:
            obs = env.reset(seed = 42)[0]
    saver.save(sess, "./my_policy_net_basic.ckpt")

2023-05-04 14:56:51.973501: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-04 14:56:51.973748: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 14:56:51.974082: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 920MX computeCapability: 5.0
coreClock: 0.993GHz coreCount: 2 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 14.92GiB/s
2023-05-04 14:56:51.974135: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-05-04 14:56:51.974203: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-05-04 14:56:51.974243: I tensorflow/stream_executor/platform/defa

In [40]:
def render_policy_net(model_path, action, X, n_max_steps=1000):
    obs = env.reset()[0]
    with tf.Session() as sess:
        saver.restore(sess, model_path)
        for step in range(n_max_steps):
            action_val = action.eval(feed_dict={X: obs.reshape(1, n_inputs)})

            obs, reward, terminated, truncated, info  = env.step(action_val[0][0])
            if terminated or truncated:
                break
    return

render_policy_net("./my_policy_net_basic.ckpt", action, X)

INFO:tensorflow:Restoring parameters from ./my_policy_net_basic.ckpt


2023-05-04 15:01:26.076773: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-04 15:01:26.076990: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 15:01:26.077262: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 920MX computeCapability: 5.0
coreClock: 0.993GHz coreCount: 2 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 14.92GiB/s
2023-05-04 15:01:26.077301: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-05-04 15:01:26.077346: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-05-04 15:01:26.077370: I tensorflow/stream_executor/platform/defa

# Learning better policy

In [66]:
reset_graph()

n_inputs = 4
n_hidden = 4
n_outputs = 1

learning_rate = 0.01

initializer = tf.variance_scaling_initializer()

X = tf.placeholder(tf.float32, shape = [None, n_inputs])

hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden, n_outputs)
outputs = tf.nn.sigmoid(logits) # probability of action 0 (left)

p_left_and_right = tf.concat(axis = 1, values=[outputs, 1 - outputs])
action = tf.multinomial(tf.log(p_left_and_right), num_samples = 1)

y = 1. - tf.to_float(action)
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)
grads_and_vars = optimizer.compute_gradients(cross_entropy)
gradients = [grad for grad, variable in grads_and_vars]
gradients_placeholders = []
grads_and_vars_feed = []
for grad, variable in grads_and_vars:
    gradients_placeholder = tf.placeholder(tf.float32, shape=grad.get_shape())
    gradients_placeholders.append(gradients_placeholder)
    grads_and_vars_feed.append((gradients_placeholder, variable))
training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [56]:
def discount_rewards(rewards, discount_rate):
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    all_discounted_rewards = [discount_rewards(rewards, discount_rate) for rewards in all_rewards]
    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    ret = [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]
    return ret

discount_rewards([10, 0, -50], discount_rate=0.8)

array([-22., -40., -50.])

In [57]:
n_iterations = 250
n_max_steps = 1000
n_games_per_update = 10
save_iterations = 10
discount_rate = 0.95

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        all_rewards = []
        all_gradients = []
        for game in range(n_games_per_update):
            current_rewards = []
            current_gradients = []
            obs = env.reset()[0]
            for step in range(n_max_steps):
                action_val, gradients_val = sess.run([action, gradients], feed_dict={X: obs.reshape(1, n_inputs)})
                obs, reward, terminated, truncated, info  = env.step(action_val[0][0])
                current_rewards.append(reward)
                current_gradients.append(gradients_val)
                if terminated or truncated:
                    break;
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)
        
        all_rewards = discount_and_normalize_rewards(all_rewards, discount_rate)
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradients_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                    for game_index, rewards in enumerate(all_rewards)
                                    for step, reward in enumerate(rewards)], axis = 0)
            feed_dict[grad_placeholder] = mean_gradients
        sess.run(training_op, feed_dict=feed_dict)
        if iteration % save_iterations == 0:
            saver.save(sess, "./my_policy_net_pg.ckpt")


2023-05-04 17:50:30.713954: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-04 17:50:30.714204: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 17:50:30.714999: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 920MX computeCapability: 5.0
coreClock: 0.993GHz coreCount: 2 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 14.92GiB/s
2023-05-04 17:50:30.715055: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-05-04 17:50:30.715125: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-05-04 17:50:30.715163: I tensorflow/stream_executor/platform/defa

In [70]:
render_policy_net("./my_policy_net_pg.ckpt", action, X, n_max_steps=10000)

INFO:tensorflow:Restoring parameters from ./my_policy_net_pg.ckpt


2023-05-04 23:30:40.620239: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-04 23:30:40.620483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-04 23:30:40.620937: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 920MX computeCapability: 5.0
coreClock: 0.993GHz coreCount: 2 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 14.92GiB/s
2023-05-04 23:30:40.620981: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2023-05-04 23:30:40.621027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2023-05-04 23:30:40.621051: I tensorflow/stream_executor/platform/defa

In [71]:
env.close()

In [61]:
print(f"Mean: {np.mean(totals)}; STD: {np.std(totals)}; Min: {np.min(totals)}; Max: {np.max(totals)}")

Mean: 39.4; STD: 8.73155198117723; Min: 26.0; Max: 58.0
